# Landhills Winery Case Study

## Model Development

### Step 1. Create Sets, Parameters, and Data

In the first step of the Landhills Winery Example, we create sets describing the genric objects of interest in the problem, declare generic parameters to hold problem relevant data, and add a data section to describe data for a particular instance of the problem.

### Step 2. Define Decision Variables

### Step 3. Create Objective Function

Now we dive into building linear arithmetic expressions for the key elements of the production model. The first of these is the objective function.  The model will run correctly, but obviously produce a rediculously large estimate of the possible profit because no problem constraints have yet been added to the model.

### Step 4. Add Diagnostic Statements

One of the challenges of developing a MathProg model is to debug the code as one moves through the development process. As this point we are generating solutions, so we will now add print statements to present the solution in a compact and meaningful form.

### Step 5. Add Constraints

In [2]:
%%script glpsol -m /dev/stdin -o /dev/stdout -y display.txt --out output

set WINES;                        # set of wines
set GRAPES;                       # set of grapes

param gVar{GRAPES} symbolic;      # variety
param gYear{GRAPES};              # vintage
param gLoc{GRAPES} symbolic;      # location
param gAcid{GRAPES};              # acid content [g/100 ml]
param gSugar{GRAPES};             # sugar content [%]
param gAlcohol{GRAPES};           # alcohol content [%]
param gCost{GRAPES};              # cost [$/bottle]
param gAvail{GRAPES};             # bottles available [bottles]

param wVar{WINES} symbolic;       # variety (empty if non-varietal)
param wYear{WINES};               # vintage (0 if non-vintage)       
param wLoc{WINES} symbolic;       # location if labeled, else empty
param wAcid{WINES};               # max acid content [g/100 ml]
param wSugar{WINES};              # max sugar content [%]
param wAlcoholMax{WINES};         # max alcohol content [%]
param wAlcoholMin{WINES};         # min alcohol content [%]
param wPrice{WINES};              # price [#/bottle]

var x{WINES,GRAPES} >= 0, <= 200000;   # allocation of grapes to wines
var wProd{WINES}    >= 0, <= 400000;            # total production of each wine type
var gCons{GRAPES}   >= 0, <= 200000;            # total consumption of each grape type
var Revenue >= 0;
var Cost >= 0;

# objective is maximize net profit
maximize Profit: Revenue - Cost;
    
# Model Definitions
s.t. rev : Revenue = sum{w in WINES} wPrice[w]*wProd[w];
s.t. cst : Cost = sum{g in GRAPES} gCost[g]*gCons[g];
    
# production and consumption
s.t. prod {w in WINES} : wProd[w] = sum {g in GRAPES} x[w,g];
s.t. cons {g in GRAPES} : gCons[g] = sum {w in WINES} x[w,g];
    
# Production Contraints
    
# consumption limited to the grapes available
s.t. avail {g in GRAPES} : gCons[g] <= gAvail[g];
    
# Product Quality Constraints
    
# acid constraints
s.t. acid {w in WINES} : sum {g in GRAPES} x[w,g]*gAcid[g] <= wAcid[w]*wProd[w];
    
# sugar constraints
s.t. sugar {w in WINES} : sum {g in GRAPES} x[w,g]*gSugar[g] <= wSugar[w]*wProd[w];
    
# Federal Regulations

# alcohol
s.t. AlcoholMin {w in WINES} : sum {g in GRAPES} x[w,g]*gAlcohol[g] >= 10.0*wProd[w];
s.t. AlcoholMax {w in WINES} : sum {g in GRAPES} x[w,g]*gAlcohol[g] <= 15.0*wProd[w];

# a labeled varietal wine must have 75% of grapes from that varietal
s.t. varietal {w in WINES} : 
    sum {g in GRAPES : gVar[g] == wVar[w]} x[w,g] >= 0.75*wProd[w];

# a labeled vintage wine must have 95% of grapes from that vintage
s.t. vintage {w in WINES : wYear[w] >= 1000} : 
    sum {g in GRAPES : gYear[g] == wYear[w]} x[w,g] >= 0.95*wProd[w];
    
# a labeled viticulture wine must have 85% of grapes from that location
s.t. viticulture {w in WINES : length(wLoc[w]) >= 1} : 
    sum {g in GRAPES : gLoc[g] == wLoc[w]} x[w,g] >= 0.85*wProd[w];
    
# Solution and Display
    
solve;

printf "Revenue = %10.2f\n", Revenue;
printf "   Cost = %10.2f\n", Cost;
printf " Profit = %10.2f\n\n", Profit;

printf "%9s   %10s", 'Wine','Production';
printf "    %6s", "Acid";
printf "    %6s", "Sugar";
printf "  %6s", "Alcohol%";
printf "  %6s", "Varietal";
printf "   %6s", "Vintage";
printf "    %6s\n", "Viti";
for {w in WINES} {
    printf "%9s", w;
    printf "    %9.2f", wProd[w];
    for {{0} : wProd[w] >= 0.1} {
        printf "    %6.2f", sum{g in GRAPES} gAcid[g]*x[w,g]/wProd[w];
        printf "    %6.2f", sum{g in GRAPES} gSugar[g]*x[w,g]/wProd[w];
        printf "    %6.2f", sum{g in GRAPES} gAlcohol[g]*x[w,g]/wProd[w];
        printf "    %6.2f", sum{g in GRAPES : gVar[g]==wVar[w]} x[w,g]/wProd[w];
        printf "    %6.2f", sum{g in GRAPES : gYear[g]==wYear[w]} x[w,g]/wProd[w];
        printf "    %6.2f", sum{g in GRAPES : gLoc[g]==wLoc[w]} x[w,g]/wProd[w];
    }
    printf "\n";
}

printf "\n%9s    %9s  %8s\n", 'GRAPE','Consumed','Available';
printf {g in GRAPES} "%9s    %9.1f %9.1f\n", g, gCons[g], gAvail[g];

printf "\n         ";
printf {g in GRAPES} "   %9s", g;
for {w in WINES} {
    printf "\n%9s",w;
    printf {g in GRAPES} "   %9.1f", x[w,g];
}
printf "\n";

data;

param : GRAPES : gVar      gYear  gLoc  gAcid  gSugar gAlcohol   gAvail  gCost:=
        gSB11   'Cabernet'  2011   'SB'   0.35    0.12     13.5    50000   2.35
        gSL10   'Cabernet'  2010   'SLO'  0.75    0.25     15.3    60000   2.60
        gSL11   'Cabernet'  2011   'SLO'  0.55    0.30     11.5    30000   2.10
        gSB10   'Merlot'    2010   'SB'   0.25    0.08     15.7   200000   1.55 ;

param : WINES : wVar       wYear  wLoc  wAcid  wSugar  wPrice :=
        wSB11  'Cabernet'   2011   'SB'    0.7     0.2   9.00
        wSL10  'Cabernet'   2010   'SLO'   0.7     0.2   9.00
        wSL11  'Cabernet'   2011   'SLO'   0.7     0.2   9.00
        wCab   'Cabernet'      0   ''      0.7     0.3   5.50
        wMer   'Merlot'        0   ''      0.3     1.0   2.95 ;

end;


In [3]:
print output

GLPSOL: GLPK LP/MIP Solver, v4.52
Parameter(s) specified in the command line:
 -m /dev/stdin -o /dev/stdout -y display.txt
Reading model section from /dev/stdin...
Reading data section from /dev/stdin...
126 lines were read
Generating Profit...
Generating rev...
Generating cst...
Generating prod...
Generating cons...
Generating avail...
Generating acid...
Generating sugar...
Generating AlcoholMin...
Generating AlcoholMax...
Generating varietal...
Generating vintage...
Generating viticulture...
Model has been successfully generated
GLPK Simplex Optimizer, v4.52
47 rows, 31 columns, 202 non-zeros
Preprocessing...
40 rows, 25 columns, 181 non-zeros
Scaling...
 A: min|aij| =  8.000e-02  max|aij| =  1.570e+01  ratio =  1.962e+02
GM: min|aij| =  4.751e-01  max|aij| =  2.105e+00  ratio =  4.430e+00
EQ: min|aij| =  2.257e-01  max|aij| =  1.000e+00  ratio =  4.430e+00
Constructing initial basis...
Size of triangular part is 40
*     0: obj =  -0.000000000e+00  infeas =  0.000e+00 (0)
*    17: o

In [4]:
print(open('display.txt')).read()

Revenue = 1343133.19
   Cost =  533221.80
 Profit =  809911.39

     Wine   Production      Acid     Sugar  Alcohol%  Varietal   Vintage      Viti
    wSB11     52631.58      0.34      0.12     13.61      0.95      0.95      1.00
    wSL10         0.00
    wSL11         0.00
     wCab     93061.22      0.60      0.21     15.00      0.75      0.00      0.00
     wMer    121224.49      0.30      0.12     15.00      0.83      0.00      0.00

    GRAPE     Consumed  Available
    gSB11      50000.0   50000.0
    gSL10      60000.0   60000.0
    gSL11      30000.0   30000.0
    gSB10     126917.3  200000.0

                gSB11       gSL10       gSL11       gSB10
    wSB11     50000.0         0.0         0.0      2631.6
    wSL10         0.0         0.0         0.0         0.0
    wSL11         0.0         0.0         0.0         0.0
     wCab         0.0     60000.0      9795.9     23265.3
     wMer         0.0         0.0     20204.1    101020.4



## Exercises

1. Which blending grapes are in excess?

2. Suppose you had purchased the blending all of the blending grapes that were available to you. (The current model assumes that you only purchase those that you actually use.)  How would the solution change?

3. What is the maximum you should spend for additional Cabernet Sauvignon grapes harvested in Santa Barbara in 2011? 

4. How does the production schedule change if only 50,000 bottles of the 2011 Santa Barbara Cabernet could be sold? 

5. Suppose you could raise the price of the non-vintage Cabernet from \$5.50 to high values through a marketing campaign. The marketing campaign would cost $100,000.  What new price would have to be acheived in order to make the it a worthwhile project?

## Solutions to Exercises

### Solution 1.

### Solution 2.

### Solution 3.

In [9]:
%%script glpsol -m /dev/stdin -o /dev/stdout -y display.txt --out output

set WINES;                        # set of wines
set GRAPES;                       # set of grapes

param gVar{GRAPES} symbolic;      # variety
param gYear{GRAPES};              # vintage
param gLoc{GRAPES} symbolic;      # location
param gAcid{GRAPES};              # acid content [g/100 ml]
param gSugar{GRAPES};             # sugar content [%]
param gAlcohol{GRAPES};           # alcohol content [%]
param gCost{GRAPES};              # cost [$/bottle]
param gAvail{GRAPES};             # bottles available [bottles]

param wVar{WINES} symbolic;       # variety (empty if non-varietal)
param wYear{WINES};               # vintage (0 if non-vintage)       
param wLoc{WINES} symbolic;       # location if labeled, else empty
param wAcid{WINES};               # max acid content [g/100 ml]
param wSugar{WINES};              # max sugar content [%]
param wAlcoholMax{WINES};         # max alcohol content [%]
param wAlcoholMin{WINES};         # min alcohol content [%]
param wPrice{WINES};              # price [#/bottle]

var x{WINES,GRAPES} >= 0, <= 200000;   # allocation of grapes to wines
var wProd{WINES}    >= 0, <= 400000;            # total production of each wine type
var gCons{GRAPES}   >= 0, <= 200000;            # total consumption of each grape type
var Revenue >= 0;
var Cost >= 0;

# objective is maximize net profit
maximize Profit: Revenue - Cost;
    
# Model Definitions
s.t. rev : Revenue = sum{w in WINES} wPrice[w]*wProd[w];
s.t. cst : Cost = sum{g in GRAPES} gCost[g]*gCons[g];
    
# production and consumption
s.t. prod {w in WINES} : wProd[w] = sum {g in GRAPES} x[w,g];
s.t. cons {g in GRAPES} : gCons[g] = sum {w in WINES} x[w,g];
    
# Production Contraints
    
# consumption limited to the grapes available
s.t. avail {g in GRAPES} : gCons[g] <= gAvail[g];
s.t. demand: 
    
# Product Quality Constraints
    
# acid constraints
s.t. acid {w in WINES} : sum {g in GRAPES} x[w,g]*gAcid[g] <= wAcid[w]*wProd[w];
    
# sugar constraints
s.t. sugar {w in WINES} : sum {g in GRAPES} x[w,g]*gSugar[g] <= wSugar[w]*wProd[w];
    
# Federal Regulations

# alcohol
s.t. AlcoholMin {w in WINES} : sum {g in GRAPES} x[w,g]*gAlcohol[g] >= 10.0*wProd[w];
s.t. AlcoholMax {w in WINES} : sum {g in GRAPES} x[w,g]*gAlcohol[g] <= 15.0*wProd[w];

# a labeled varietal wine must have 75% of grapes from that varietal
s.t. varietal {w in WINES} : 
    sum {g in GRAPES : gVar[g] == wVar[w]} x[w,g] >= 0.75*wProd[w];

# a labeled vintage wine must have 95% of grapes from that vintage
s.t. vintage {w in WINES : wYear[w] >= 1000} : 
    sum {g in GRAPES : gYear[g] == wYear[w]} x[w,g] >= 0.95*wProd[w];
    
# a labeled viticulture wine must have 85% of grapes from that location
s.t. viticulture {w in WINES : length(wLoc[w]) >= 1} : 
    sum {g in GRAPES : gLoc[g] == wLoc[w]} x[w,g] >= 0.85*wProd[w];
    
# Solution and Display
    
solve;

printf "Revenue = %10.2f\n", Revenue;
printf "   Cost = %10.2f\n", Cost;
printf " Profit = %10.2f\n\n", Profit;

printf "%9s   %10s", 'Wine','Production';
printf "    %6s", "Acid";
printf "    %6s", "Sugar";
printf "  %6s", "Alcohol%";
printf "  %6s", "Varietal";
printf "   %6s", "Vintage";
printf "    %6s\n", "Viti";
for {w in WINES} {
    printf "%9s", w;
    printf "    %9.2f", wProd[w];
    for {{0} : wProd[w] >= 0.1} {
        printf "    %6.2f", sum{g in GRAPES} gAcid[g]*x[w,g]/wProd[w];
        printf "    %6.2f", sum{g in GRAPES} gSugar[g]*x[w,g]/wProd[w];
        printf "    %6.2f", sum{g in GRAPES} gAlcohol[g]*x[w,g]/wProd[w];
        printf "    %6.2f", sum{g in GRAPES : gVar[g]==wVar[w]} x[w,g]/wProd[w];
        printf "    %6.2f", sum{g in GRAPES : gYear[g]==wYear[w]} x[w,g]/wProd[w];
        printf "    %6.2f", sum{g in GRAPES : gLoc[g]==wLoc[w]} x[w,g]/wProd[w];
    }
    printf "\n";
}

printf "\n%9s    %9s  %8s\n", 'GRAPE','Consumed','Available';
printf {g in GRAPES} "%9s    %9.1f %9.1f\n", g, gCons[g], gAvail[g];

printf "\n         ";
printf {g in GRAPES} "   %9s", g;
for {w in WINES} {
    printf "\n%9s",w;
    printf {g in GRAPES} "   %9.1f", x[w,g];
}
printf "\n";

printf {g in GRAPES} "  %9s  %6.2f\n", g, avail[g].dual;

data;

param : GRAPES : gVar      gYear  gLoc  gAcid  gSugar gAlcohol   gAvail  gCost:=
        gSB11   'Cabernet'  2011   'SB'   0.35    0.12     13.5    50000   2.35
        gSL10   'Cabernet'  2010   'SLO'  0.75    0.25     15.3    60000   2.60
        gSL11   'Cabernet'  2011   'SLO'  0.55    0.30     11.5    30000   2.10
        gSB10   'Merlot'    2010   'SB'   0.25    0.08     15.7   200000   1.55 ;

param : WINES : wVar       wYear  wLoc  wAcid  wSugar  wPrice :=
        wSB11  'Cabernet'   2011   'SB'    0.7     0.2   9.00
        wSL10  'Cabernet'   2010   'SLO'   0.7     0.2   9.00
        wSL11  'Cabernet'   2011   'SLO'   0.7     0.2   9.00
        wCab   'Cabernet'      0   ''      0.7     0.3   5.50
        wMer   'Merlot'        0   ''      0.3     1.0   2.95 ;

end;


In [10]:
print open('display.txt').read()

Revenue = 1343133.19
   Cost =  533221.80
 Profit =  809911.39

     Wine   Production      Acid     Sugar  Alcohol%  Varietal   Vintage      Viti
    wSB11     52631.58      0.34      0.12     13.61      0.95      0.95      1.00
    wSL10         0.00
    wSL11         0.00
     wCab     93061.22      0.60      0.21     15.00      0.75      0.00      0.00
     wMer    121224.49      0.30      0.12     15.00      0.83      0.00      0.00

    GRAPE     Consumed  Available
    gSB11      50000.0   50000.0
    gSL10      60000.0   60000.0
    gSL11      30000.0   30000.0
    gSB10     126917.3  200000.0

                gSB11       gSL10       gSL11       gSB10
    wSB11     50000.0         0.0         0.0      2631.6
    wSL10         0.0         0.0         0.0         0.0
    wSL11         0.0         0.0         0.0         0.0
     wCab         0.0     60000.0      9795.9     23265.3
     wMer         0.0         0.0     20204.1    101020.4
      gSB11    7.04
      gSL10    3.71
  

### Solution 4.

In [13]:
%%script glpsol -m /dev/stdin -o /dev/stdout -y display.txt --out output

set WINES;                        # set of wines
set GRAPES;                       # set of grapes

param gVar{GRAPES} symbolic;      # variety
param gYear{GRAPES};              # vintage
param gLoc{GRAPES} symbolic;      # location
param gAcid{GRAPES};              # acid content [g/100 ml]
param gSugar{GRAPES};             # sugar content [%]
param gAlcohol{GRAPES};           # alcohol content [%]
param gCost{GRAPES};              # cost [$/bottle]
param gAvail{GRAPES};             # bottles available [bottles]

param wVar{WINES} symbolic;       # variety (empty if non-varietal)
param wYear{WINES};               # vintage (0 if non-vintage)       
param wLoc{WINES} symbolic;       # location if labeled, else empty
param wAcid{WINES};               # max acid content [g/100 ml]
param wSugar{WINES};              # max sugar content [%]
param wAlcoholMax{WINES};         # max alcohol content [%]
param wAlcoholMin{WINES};         # min alcohol content [%]
param wPrice{WINES};              # price [#/bottle]
param wProdLimit{WINES} default 1000000;          # production limit

var x{WINES,GRAPES} >= 0, <= 200000;   # allocation of grapes to wines
var wProd{WINES}    >= 0, <= 400000;            # total production of each wine type
var gCons{GRAPES}   >= 0, <= 200000;            # total consumption of each grape type
var Revenue >= 0;
var Cost >= 0;

# objective is maximize net profit
maximize Profit: Revenue - Cost;
    
# Model Definitions
s.t. rev : Revenue = sum{w in WINES} wPrice[w]*wProd[w];
s.t. cst : Cost = sum{g in GRAPES} gCost[g]*gCons[g];
    
# production and consumption
s.t. prod {w in WINES} : wProd[w] = sum {g in GRAPES} x[w,g];
s.t. cons {g in GRAPES} : gCons[g] = sum {w in WINES} x[w,g];
    
# Production Contraints
    
# consumption limited to the grapes available
s.t. avail {g in GRAPES} : gCons[g] <= gAvail[g];
s.t. prodlimit {w in WINES} : wProd[w] <= wProdLimit[w];
    
# Product Quality Constraints
    
# acid constraints
s.t. acid {w in WINES} : sum {g in GRAPES} x[w,g]*gAcid[g] <= wAcid[w]*wProd[w];
    
# sugar constraints
s.t. sugar {w in WINES} : sum {g in GRAPES} x[w,g]*gSugar[g] <= wSugar[w]*wProd[w];
    
# Federal Regulations

# alcohol
s.t. AlcoholMin {w in WINES} : sum {g in GRAPES} x[w,g]*gAlcohol[g] >= 10.0*wProd[w];
s.t. AlcoholMax {w in WINES} : sum {g in GRAPES} x[w,g]*gAlcohol[g] <= 15.0*wProd[w];

# a labeled varietal wine must have 75% of grapes from that varietal
s.t. varietal {w in WINES} : 
    sum {g in GRAPES : gVar[g] == wVar[w]} x[w,g] >= 0.75*wProd[w];

# a labeled vintage wine must have 95% of grapes from that vintage
s.t. vintage {w in WINES : wYear[w] >= 1000} : 
    sum {g in GRAPES : gYear[g] == wYear[w]} x[w,g] >= 0.95*wProd[w];
    
# a labeled viticulture wine must have 85% of grapes from that location
s.t. viticulture {w in WINES : length(wLoc[w]) >= 1} : 
    sum {g in GRAPES : gLoc[g] == wLoc[w]} x[w,g] >= 0.85*wProd[w];
    
# Solution and Display
    
solve;

printf "Revenue = %10.2f\n", Revenue;
printf "   Cost = %10.2f\n", Cost;
printf " Profit = %10.2f\n\n", Profit;

printf "%9s   %10s", 'Wine','Production';
printf "    %6s", "Acid";
printf "    %6s", "Sugar";
printf "  %6s", "Alcohol%";
printf "  %6s", "Varietal";
printf "   %6s", "Vintage";
printf "    %6s\n", "Viti";
for {w in WINES} {
    printf "%9s", w;
    printf "    %9.2f", wProd[w];
    for {{0} : wProd[w] >= 0.1} {
        printf "    %6.2f", sum{g in GRAPES} gAcid[g]*x[w,g]/wProd[w];
        printf "    %6.2f", sum{g in GRAPES} gSugar[g]*x[w,g]/wProd[w];
        printf "    %6.2f", sum{g in GRAPES} gAlcohol[g]*x[w,g]/wProd[w];
        printf "    %6.2f", sum{g in GRAPES : gVar[g]==wVar[w]} x[w,g]/wProd[w];
        printf "    %6.2f", sum{g in GRAPES : gYear[g]==wYear[w]} x[w,g]/wProd[w];
        printf "    %6.2f", sum{g in GRAPES : gLoc[g]==wLoc[w]} x[w,g]/wProd[w];
    }
    printf "\n";
}

printf "\n%9s    %9s  %8s\n", 'GRAPE','Consumed','Available';
printf {g in GRAPES} "%9s    %9.1f %9.1f\n", g, gCons[g], gAvail[g];

printf "\n         ";
printf {g in GRAPES} "   %9s", g;
for {w in WINES} {
    printf "\n%9s",w;
    printf {g in GRAPES} "   %9.1f", x[w,g];
}
printf "\n";

printf {g in GRAPES} "  %9s  %6.2f\n", g, avail[g].dual;

data;

param : GRAPES : gVar      gYear  gLoc  gAcid  gSugar gAlcohol   gAvail  gCost:=
        gSB11   'Cabernet'  2011   'SB'   0.35    0.12     13.5    50000   2.35
        gSL10   'Cabernet'  2010   'SLO'  0.75    0.25     15.3    60000   2.60
        gSL11   'Cabernet'  2011   'SLO'  0.55    0.30     11.5    30000   2.10
        gSB10   'Merlot'    2010   'SB'   0.25    0.08     15.7   200000   1.55 ;

param : WINES : wVar       wYear  wLoc  wAcid  wSugar  wPrice   wProdLimit :=
        wSB11  'Cabernet'   2011   'SB'    0.7     0.2   9.00      50000
        wSL10  'Cabernet'   2010   'SLO'   0.7     0.2   9.00          .
        wSL11  'Cabernet'   2011   'SLO'   0.7     0.2   9.00          .
        wCab   'Cabernet'      0   ''      0.7     0.3   5.50          .
        wMer   'Merlot'        0   ''      0.3     1.0   2.95          . ;

end;


In [14]:
print open('display.txt').read()

Revenue = 1347831.63
   Cost =  541321.43
 Profit =  806510.20

     Wine   Production      Acid     Sugar  Alcohol%  Varietal   Vintage      Viti
    wSB11     50000.00      0.34      0.12     13.61      0.95      0.95      1.00
    wSL10         0.00
    wSL11         0.00
     wCab     95102.04      0.60      0.21     15.00      0.75      0.00      0.00
     wMer    127040.82      0.30      0.12     15.00      0.83      0.00      0.00

    GRAPE     Consumed  Available
    gSB11      50000.0   50000.0
    gSL10      60000.0   60000.0
    gSL11      30000.0   30000.0
    gSB10     132142.9  200000.0

                gSB11       gSL10       gSL11       gSB10
    wSB11     47500.0         0.0         0.0      2500.0
    wSL10         0.0         0.0         0.0         0.0
    wSL11         0.0         0.0         0.0         0.0
     wCab      2500.0     60000.0      8826.5     23775.5
     wMer         0.0         0.0     21173.5    105867.3
      gSB11    5.68
      gSL10    3.71
  

### Solution 5.

In [55]:
%%script glpsol -m /dev/stdin -o /dev/stdout -y display.txt --out output

set WINES;                        # set of wines
set GRAPES;                       # set of grapes

param gVar{GRAPES} symbolic;      # variety
param gYear{GRAPES};              # vintage
param gLoc{GRAPES} symbolic;      # location
param gAcid{GRAPES};              # acid content [g/100 ml]
param gSugar{GRAPES};             # sugar content [%]
param gAlcohol{GRAPES};           # alcohol content [%]
param gCost{GRAPES};              # cost [$/bottle]
param gAvail{GRAPES};             # bottles available [bottles]

param wVar{WINES} symbolic;       # variety (empty if non-varietal)
param wYear{WINES};               # vintage (0 if non-vintage)       
param wLoc{WINES} symbolic;       # location if labeled, else empty
param wAcid{WINES};               # max acid content [g/100 ml]
param wSugar{WINES};              # max sugar content [%]
param wAlcoholMax{WINES};         # max alcohol content [%]
param wAlcoholMin{WINES};         # min alcohol content [%]
param wPrice{WINES};              # price [#/bottle]

var x{WINES,GRAPES} >= 0, <= 200000;   # allocation of grapes to wines
var wProd{WINES}    >= 0, <= 400000;            # total production of each wine type
var gCons{GRAPES}   >= 0, <= 200000;            # total consumption of each grape type
var Revenue >= 0;
var Cost >= 0;

# objective is maximize net profit
maximize Profit: Revenue - Cost;
    
# Model Definitions
s.t. rev : Revenue = sum{w in WINES} wPrice[w]*wProd[w];
s.t. cst : Cost = sum{g in GRAPES} gCost[g]*gCons[g];
    
# production and consumption
s.t. prod {w in WINES} : wProd[w] = sum {g in GRAPES} x[w,g];
s.t. cons {g in GRAPES} : gCons[g] = sum {w in WINES} x[w,g];
    
# Production Contraints
    
# consumption limited to the grapes available
s.t. avail {g in GRAPES} : gCons[g] <= gAvail[g];
    
# Product Quality Constraints
    
# acid constraints
s.t. acid {w in WINES} : sum {g in GRAPES} x[w,g]*gAcid[g] <= wAcid[w]*wProd[w];
    
# sugar constraints
s.t. sugar {w in WINES} : sum {g in GRAPES} x[w,g]*gSugar[g] <= wSugar[w]*wProd[w];
    
# Federal Regulations

# alcohol
s.t. AlcoholMin {w in WINES} : sum {g in GRAPES} x[w,g]*gAlcohol[g] >= 10.0*wProd[w];
s.t. AlcoholMax {w in WINES} : sum {g in GRAPES} x[w,g]*gAlcohol[g] <= 15.0*wProd[w];

# a labeled varietal wine must have 75% of grapes from that varietal
s.t. varietal {w in WINES} : 
    sum {g in GRAPES : gVar[g] == wVar[w]} x[w,g] >= 0.75*wProd[w];

# a labeled vintage wine must have 95% of grapes from that vintage
s.t. vintage {w in WINES : wYear[w] >= 1000} : 
    sum {g in GRAPES : gYear[g] == wYear[w]} x[w,g] >= 0.95*wProd[w];
    
# a labeled viticulture wine must have 85% of grapes from that location
s.t. viticulture {w in WINES : length(wLoc[w]) >= 1} : 
    sum {g in GRAPES : gLoc[g] == wLoc[w]} x[w,g] >= 0.85*wProd[w];
    
# Solution and Display
    
solve;

printf "Revenue = %10.2f\n", Revenue;
printf "   Cost = %10.2f\n", Cost;
printf " Profit = %10.2f\n\n", Profit;

printf "%9s   %10s", 'Wine','Production';
printf "    %6s", "Acid";
printf "    %6s", "Sugar";
printf "  %6s", "Alcohol%";
printf "  %6s", "Varietal";
printf "   %6s", "Vintage";
printf "    %6s\n", "Viti";
for {w in WINES} {
    printf "%9s", w;
    printf "    %9.2f", wProd[w];
    for {{0} : wProd[w] >= 0.1} {
        printf "    %6.2f", sum{g in GRAPES} gAcid[g]*x[w,g]/wProd[w];
        printf "    %6.2f", sum{g in GRAPES} gSugar[g]*x[w,g]/wProd[w];
        printf "    %6.2f", sum{g in GRAPES} gAlcohol[g]*x[w,g]/wProd[w];
        printf "    %6.2f", sum{g in GRAPES : gVar[g]==wVar[w]} x[w,g]/wProd[w];
        printf "    %6.2f", sum{g in GRAPES : gYear[g]==wYear[w]} x[w,g]/wProd[w];
        printf "    %6.2f", sum{g in GRAPES : gLoc[g]==wLoc[w]} x[w,g]/wProd[w];
    }
    printf "\n";
}

printf "\n%9s    %9s  %8s\n", 'GRAPE','Consumed','Available';
printf {g in GRAPES} "%9s    %9.1f %9.1f\n", g, gCons[g], gAvail[g];

printf "\n         ";
printf {g in GRAPES} "   %9s", g;
for {w in WINES} {
    printf "\n%9s",w;
    printf {g in GRAPES} "   %9.1f", x[w,g];
}
printf "\n";

printf {g in GRAPES} "  %9s  %6.2f\n", g, avail[g].dual;

data;

param : GRAPES : gVar      gYear  gLoc  gAcid  gSugar gAlcohol   gAvail  gCost:=
        gSB11   'Cabernet'  2011   'SB'   0.35    0.12     13.5    50000   2.35
        gSL10   'Cabernet'  2010   'SLO'  0.75    0.25     15.3    60000   2.60
        gSL11   'Cabernet'  2011   'SLO'  0.55    0.30     11.5    30000   2.10
        gSB10   'Merlot'    2010   'SB'   0.25    0.08     15.7   200000   1.55 ;

param : WINES : wVar       wYear  wLoc  wAcid  wSugar  wPrice :=
        wSB11  'Cabernet'   2011   'SB'    0.7     0.2   9.00
        wSL10  'Cabernet'   2010   'SLO'   0.7     0.2   9.00
        wSL11  'Cabernet'   2011   'SLO'   0.7     0.2   9.00
        wCab   'Cabernet'      0   ''      0.7     0.3   6.5745615
        wMer   'Merlot'        0   ''      0.3     1.0   2.95 ;

end;


In [56]:
print open('display.txt').read()

Revenue = 1443133.20
   Cost =  533221.80
 Profit =  909911.39

     Wine   Production      Acid     Sugar  Alcohol%  Varietal   Vintage      Viti
    wSB11     52631.58      0.34      0.12     13.61      0.95      0.95      1.00
    wSL10         0.00
    wSL11         0.00
     wCab     93061.22      0.60      0.21     15.00      0.75      0.00      0.00
     wMer    121224.49      0.30      0.12     15.00      0.83      0.00      0.00

    GRAPE     Consumed  Available
    gSB11      50000.0   50000.0
    gSL10      60000.0   60000.0
    gSL11      30000.0   30000.0
    gSB10     126917.3  200000.0

                gSB11       gSL10       gSL11       gSB10
    wSB11     50000.0         0.0         0.0      2631.6
    wSL10         0.0         0.0         0.0         0.0
    wSL11         0.0         0.0         0.0         0.0
     wCab         0.0     60000.0      9795.9     23265.3
     wMer         0.0         0.0     20204.1    101020.4
      gSB11    7.04
      gSL10    5.37
  

In [24]:
print output

GLPSOL: GLPK LP/MIP Solver, v4.57
Parameter(s) specified in the command line:
 -m /dev/stdin -o /dev/stdout -y display.txt
Reading model section from /dev/stdin...
Reading data section from /dev/stdin...
128 lines were read
Generating Profit...
Generating rev...
Generating cst...
Generating prod...
Generating cons...
Generating avail...
Generating acid...
Generating sugar...
Generating AlcoholMin...
Generating AlcoholMax...
Generating varietal...
Generating vintage...
Generating viticulture...
Model has been successfully generated
glp_mpl_build_prob: row Profit; constant term -909911.39 ignored
GLPK Simplex Optimizer, v4.57
47 rows, 31 columns, 202 non-zeros
Preprocessing...
40 rows, 25 columns, 181 non-zeros
Scaling...
 A: min|aij| =  8.000e-02  max|aij| =  1.570e+01  ratio =  1.962e+02
GM: min|aij| =  4.751e-01  max|aij| =  2.105e+00  ratio =  4.430e+00
EQ: min|aij| =  2.257e-01  max|aij| =  1.000e+00  ratio =  4.430e+00
Constructing initial basis...
Size of triangular part is 40
*  